# data preproccessing


## import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
pd.set_option('max.rows', None)
pd.set_option('max.columns', None)


from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
%matplotlib inline

### configurations
* data_input_path -> string, filepath to data to be read in

* grouping -> boolean, if set to True them many sites are grouped
* reduced_grouping -> boolean, if set to True then less sites grouped, only bedrock sites 'WB' and 'BX are grouped into one class and superficial sites 'SV' and 'SE' are grouped into one class.
* raw -> boolean, if set to True then no grouping done
* drop_semi_bedrock ->  True|False, if set to True then some bedrock sites deemed to be semi-bedrock sites are not used for classification



In [2]:
data_input_path = '../data/raw_data.csv'


grouping = False

grouping_reduced = True
raw = False
drop_semi_bedrock = False


In [3]:
%store -r X_test_labeled_df

### Import data

In [4]:
my_data = pd.read_csv(data_input_path)

In [5]:
my_data.Geology.value_counts()

Bedrock        808
Superficial    435
Artefacts      363
Name: Geology, dtype: int64

### features

In [6]:
my_data.columns.values[9:-1]

array(['Li7', 'Be9', 'B11', 'Mg24', 'Al27', 'Si28', 'P31', 'S33', 'K39',
       'Ca42', 'Sc45', 'Ti47', 'V51', 'Cr52', 'Mn55', 'Fe56', 'Co59',
       'Ni60', 'Cu63', 'Zn68', 'Ga69', 'Ge72', 'As75', 'Rb85', 'Sr88',
       'Y89', 'Zr90', 'Nb93', 'Mo95', 'Cd111', 'In115', 'Sn118', 'Cs133',
       'Ba137', 'La139', 'Ce140', 'Pr141', 'Nd146', 'Sm147', 'Eu153',
       'Gd157', 'Tb159', 'Dy163', 'Ho165', 'Er166', 'Tm169', 'Yb172',
       'Lu175', 'Hf178', 'Ta181', 'Pb208', 'Th232'], dtype=object)

### raw sample names including sample sites and artefacts

In [7]:
my_data['Site'].unique()

array(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG', 'WB',
       'BX', 'PF', 'BM', 'WH', 'SQ', 'BP', 'WN', 'BH', 'PH', 'LB', 'AB',
       'LV', 'BR', 'KY', 'BF', 'ST', 'SH', 'CF', 'BG', 'AC', 'CR', 'GH',
       'PX', 'WF', 'DH', 'NMAG_Gold', 'NMW_Gold', 'NMWGwern', 'UBSS',
       'Cefn', 'Stockley', 'Pucha', 'Woodbury', 'Pimple', 'Wellington',
       'Lyonshall', 'SymondsYatE', 'Madawg', nan], dtype=object)

### define functions for making target classes for classification

In [8]:

def make_classes_grouped(row):
    if row['Geology'] == 'Bedrock':
        if row['Site'] == 'WB' or row['Site'] == 'BX':
            return('WB_BX')
        elif row['Site'] == 'BC' or row['Site'] == 'CS':
            return('BC_CS')
        elif row['Site'] == 'SQ' or row['Site'] == 'BP':
            return('SQ_BP')
        else:
            return(row['Site'])
    elif row['Geology'] == 'Superficial':
        if row['Region'] == 'SV' or row['Region'] == 'SE':
            return('SV_SE')
        else:
            return(row['Region'])
        
def make_classes_grouped_reduced(row):
    if row['Geology'] == 'Bedrock':
        if row['Site'] == 'WB' or row['Site'] == 'BX':
            return('WB_BX')
        else:
            return(row['Site'])
    elif row['Geology'] == 'Superficial':
        if row['Region'] == 'SV' or row['Region'] == 'SE':
            return('SV_SE')
        else:
            return(row['Region'])

def make_classes_raw(row):
    if row['Geology'] == 'Bedrock':
        return(row['Site'])
    elif row['Geology'] == 'Superficial':
        return(row['Region'])



     

### targets for classification are made

In [9]:
my_data['class'] = 'init'   

if grouping:
    my_data['class'] = my_data.apply(make_classes_grouped, axis = 1)
elif grouping_reduced:
    my_data['class'] = my_data.apply(make_classes_grouped_reduced, axis = 1)
elif raw:
    my_data['class'] = my_data.apply(make_classes_raw, axis = 1)

    


In [10]:
if drop_semi_bedrock:
    my_data = my_data[(my_data['class'] != 'BM') & (my_data['class'] != 'BC') & (my_data['class'] != 'BP') ]

### remove '<' signs and commas from feature values

In [11]:
for column_name in my_data.columns.values[9:-1]:
    def fill_less_than(row):
        if 'DL' in  str(row[column_name]):
            return(np.nan)
        if '<' in str(row[column_name]):
            return(float(row[column_name].replace('<', '').replace(',', '')))
        else:
            return(float(row[column_name]))
    my_data[column_name] = my_data.swifter.apply(fill_less_than, axis = 1)

Pandas Apply: 100%|██████████| 1627/1627 [00:00<00:00, 14001.98it/s]


### remove rows where there are all element abundances are na values 

In [12]:
my_data = my_data.dropna(subset=my_data.columns.values[9:-1], how = 'all' , axis = 0)

### Impute na values with feature mean

In [13]:
for column_name in my_data.columns.values[9:-1]:
    my_data[column_name] = my_data[column_name].fillna(my_data[column_name].mean()) 

### Outliers defined as any values that exceed 2 standard deviations from the mean, such values are changed to the mean for that variable

In [14]:
std_dict = {}
mean_dict = {}
median_dict = {}

for col in my_data.columns.values[9:-1]:
    std_dict[col] = my_data[col].std()
    
for col in my_data.columns.values[9:-1]:
    mean_dict[col] = my_data[col].mean()
    
for col in my_data.columns.values[9:-1]:
    median_dict[col] = my_data[col].median()
    

In [15]:
for col_name in my_data.columns.values[9:-1]:
    def impute_outliers(row):
        if np.abs(row[col_name] - mean_dict[col_name]) > 2*(std_dict[col_name]):
            return(mean_dict[col_name])
        else:
            return(row[col_name])
    my_data[col_name]= my_data.swifter.apply(impute_outliers, axis = 1)

Pandas Apply: 100%|██████████| 1596/1596 [00:00<00:00, 17424.18it/s]


### split data into 'train_data' and 'test_data', the former consists of samples from known geological sites and the latter from flint artefacts fow which the original geological source site is unknown and to be predicted.

In [16]:
my_data.isna().any()

Analysis     False
Geology      False
Province      True
Region        True
Site         False
SubSite       True
Formation     True
Band          True
Nodule       False
Li7          False
Be9          False
B11          False
Mg24         False
Al27         False
Si28         False
P31          False
S33          False
K39          False
Ca42         False
Sc45         False
Ti47         False
V51          False
Cr52         False
Mn55         False
Fe56         False
Co59         False
Ni60         False
Cu63         False
Zn68         False
Ga69         False
Ge72         False
As75         False
Rb85         False
Sr88         False
Y89          False
Zr90         False
Nb93         False
Mo95         False
Cd111        False
In115        False
Sn118        False
Cs133        False
Ba137        False
La139        False
Ce140        False
Pr141        False
Nd146        False
Sm147        False
Eu153        False
Gd157        False
Tb159        False
Dy163        False
Ho165       

In [17]:
train_data = my_data[(my_data['Geology']== 'Bedrock') | (my_data['Geology'] == 'Superficial')]
test_data = my_data[my_data['Geology']=='Artefacts']


### label encode the class to be predicted

### 'training_data' is split into two datasets, one consisting of samples from superficial sites and one containing samples from bedrock sites.

In [18]:
train_data_bedrock = train_data[train_data['Geology'] == 'Bedrock']
train_data_superficial = train_data[train_data['Geology'] == 'Superficial']

### four datasets are created, one containing all train data (bedrock and superficial types), one containing just bedrock types, one containing just superficial types and one containing the artefacts

In [19]:
element_data_train = train_data[train_data.columns.values[9:-1]]
element_data_train_bedrock = train_data_bedrock[train_data.columns.values[9:-1]]
element_data_train_superficial = train_data_superficial[train_data.columns.values[9:-1]]
element_data_test = test_data[test_data.columns.values[9:-1]]
element_data_everything = my_data[my_data.columns.values[9:-1]]

In [20]:
element_data_everything.head()

,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238
0,15.63,0.12,48.36,33.359881,943.71,464944.180000,50.280000,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05
1,11.50,0.09,44.77,33.359881,1077.11,465010.940000,70.910000,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04
2,20.05,0.06,44.88,42.700000,620.21,465295.410000,104.470000,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05
3,11.16,0.73,47.06,33.359881,1143.19,462284.527976,2309.325714,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03
4,17.71,0.32,48.26,33.520000,547.22,465027.110000,44.440000,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09


### features are standardised

In [21]:
my_scaler_train = StandardScaler()
my_scaler_train_bedrock = StandardScaler()
my_scaler_train_superficial = StandardScaler()
my_scaler_test = StandardScaler()
my_scaler_everything = StandardScaler()

In [22]:
element_data_train_scaled = my_scaler_train.fit_transform(element_data_train)
element_data_train_bedrock_scaled = my_scaler_train_bedrock.fit_transform(element_data_train_bedrock)
element_data_train_superficial_scaled = my_scaler_train_superficial.fit_transform(element_data_train_superficial)
element_data_test_scaled = my_scaler_test.fit_transform(element_data_test)
element_data_everything_scaled = my_scaler_everything.fit_transform(element_data_everything)

### The four datasets are transformed using Principal component analysis 

In [23]:
my_pca_train = PCA(n_components=element_data_train_scaled.shape[1])
my_pca_train_bedrock = PCA(n_components=element_data_train_bedrock_scaled.shape[1])
my_pca_train_superficial = PCA(n_components=element_data_train_superficial_scaled.shape[1])
my_pca_test = PCA(n_components=element_data_test_scaled.shape[1])
my_pca_everything = PCA(n_components=element_data_everything_scaled.shape[1])

element_data_train_pca = my_pca_train.fit_transform(element_data_train_scaled)
element_data_train_bedrock_pca = my_pca_train_bedrock.fit_transform(element_data_train_bedrock_scaled)
element_data_train_superficial_pca = my_pca_train_superficial.fit_transform(element_data_train_superficial_scaled)
element_data_test_pca = my_pca_test.fit_transform(element_data_test_scaled)
element_data_everything_pca = my_pca_everything.fit_transform(element_data_everything)

### print details of PCA

In [24]:
print(my_pca_train)

PCA(copy=True, iterated_power='auto', n_components=53, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


### the principal components for the four datasets are put into dataframes

In [25]:
no_PCs = element_data_train_scaled.shape[1]
PC_names = []
for i in range(0, no_PCs):
    number = i + 1
    column_name = 'PC' + str(number)
    PC_names.append(column_name)

In [26]:
PC_df_train = pd.DataFrame(data = element_data_train_pca, columns = PC_names)
PC_df_bedrock_train = pd.DataFrame(data = element_data_train_bedrock_pca, columns = PC_names)
PC_df_superficial_train = pd.DataFrame(data = element_data_train_superficial_pca, columns = PC_names)
PC_df_test = pd.DataFrame(data = element_data_test_pca, columns = PC_names)
PC_df_everything = pd.DataFrame(data = element_data_everything_pca, columns = PC_names)

### T-Distributed Stochastic Neighbour Embedding is done on the four datasets

In [27]:
#my_tsne_train = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_train_scaled)
#my_tsne_bedrock_train = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_train_bedrock_scaled)
#my_tsne_superficial_train = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_train_superficial_scaled)
#my_tsne_test = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_test_scaled)
my_tsne_everything = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_everything_scaled)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1596 samples in 0.003s...
[t-SNE] Computed neighbors for 1596 samples in 0.441s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1596
[t-SNE] Computed conditional probabilities for sample 1596 / 1596
[t-SNE] Mean sigma: 1.939194
[t-SNE] Computed conditional probabilities in 0.134s
[t-SNE] Iteration 50: error = 73.5257721, gradient norm = 0.0669982 (50 iterations in 9.040s)
[t-SNE] Iteration 100: error = 73.3519669, gradient norm = 0.0676636 (50 iterations in 5.128s)
[t-SNE] Iteration 150: error = 73.0018845, gradient norm = 0.0513625 (50 iterations in 2.816s)
[t-SNE] Iteration 200: error = 73.1751099, gradient norm = 0.0485516 (50 iterations in 2.755s)
[t-SNE] Iteration 250: error = 73.0546494, gradient norm = 0.0426513 (50 iterations in 2.542s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 73.054649
[t-SNE] Iteration 300: error = 1.4018887, gradient norm = 0.0006252 (50 iterations in 3.016s)

### the t-SNE dimensions for the four datasets are put into dataframes

In [28]:
#tsne_df_train = pd.DataFrame(data = my_tsne_train, columns = ['tsne1', 'tsne2', 'tsne3'])
#tsne_df_bedrock_train = pd.DataFrame(data = my_tsne_bedrock_train, columns = ['tsne1', 'tsne2', 'tsne3'])
#tsne_df_superficial_train = pd.DataFrame(data = my_tsne_superficial_train, columns = ['tsne1', 'tsne2', 'tsne3'])
#tsne_df_test = pd.DataFrame(data = my_tsne_test, columns = ['tsne1', 'tsne2', 'tsne3'])
tsne_df_everything = pd.DataFrame(data = my_tsne_everything, columns = ['tsne1', 'tsne2', 'tsne3'])

In [29]:
bedrock_data = train_data[train_data['Geology'] == 'Bedrock']
superficial_data = train_data[train_data['Geology'] == 'Superficial'].reset_index(drop = True)
artefact_data = pd.concat([test_data.reset_index(drop = True), X_test_labeled_df['inlierLabel'].reset_index(drop = True)], axis = 1)

In [30]:
def addartefactclass(row):
    if row['Geology'] == 'Bedrock':
        return(row['class'])
    elif row['Geology'] == 'Superficial':
        return(row['class'])
    else:
        return('Artefact')
my_data['class'] = my_data.swifter.apply(addartefactclass, axis = 1)

Pandas Apply: 100%|██████████| 1596/1596 [00:00<00:00, 29583.69it/s]


In [31]:
artefact_data.head()

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class,inlierLabel
0,06_DH1_1,Artefacts,NaN,NaN,DH,NaN,NaN,NaN,DH1,2.79,0.95,93.21,27.83,635.240000,465388.29,35.07,806.55,443.100000,119.05,1.08,37.7700,0.51,5.18,3.27,35.38,0.11,1.03,11.14,20.75,0.69,1.37,0.44,0.720000,2.00,0.21,1.43,0.222487,0.08,0.207143,0.01,0.19,0.03,5.95,0.18,0.57,0.07,0.34,0.04,0.01,0.05,0.01,0.06,0.01,0.03,0.01,0.03,0.02,0.04,0.01,6.12,0.11,0.309799,None,1
1,07_DH1_2,Artefacts,NaN,NaN,DH,NaN,NaN,NaN,DH1,2.37,1.29,95.19,27.99,639.090000,465388.40,31.65,807.55,442.100000,119.74,1.09,24.3200,0.60,5.43,2.30,49.23,0.10,1.22,8.64,18.15,0.86,1.93,0.44,0.770000,1.84,0.21,1.47,0.222487,0.11,0.207143,0.01,0.19,0.03,7.51,0.19,0.53,0.06,0.24,0.07,0.02,0.05,0.01,0.05,0.01,0.02,0.00,0.01,0.00,0.04,0.01,5.57,0.09,0.309799,None,1
2,08_DH1_3,Artefacts,NaN,NaN,DH,NaN,NaN,NaN,DH1,2.46,1.33,93.63,26.43,670.480000,465378.49,25.43,837.73,444.280000,140.99,1.17,17.9300,0.62,2.61,1.40,16.46,0.10,1.01,3.44,17.54,0.79,2.14,0.42,0.710000,1.93,0.13,1.16,0.222487,0.04,0.207143,0.01,0.12,0.02,4.40,0.11,0.20,0.03,0.08,0.04,0.01,0.03,0.00,0.02,0.01,0.01,0.00,0.01,0.00,0.04,0.03,3.51,0.08,0.309799,None,1
3,09_DH2_1,Artefacts,NaN,NaN,DH,NaN,NaN,NaN,DH2,18.47,0.11,63.80,31.98,620.082431,464555.55,24.63,750.62,383.119348,141.70,1.19,72.6500,1.52,2.87,1.09,16.76,0.09,1.07,1.10,14.33,0.75,1.59,0.46,1.620000,4.29,0.20,1.45,0.250000,0.03,0.220000,0.01,0.19,0.04,12.48,0.23,0.40,0.06,0.25,0.06,0.02,0.05,0.01,0.05,0.01,0.02,0.00,0.05,0.00,0.05,0.01,0.54,0.09,0.080000,None,1
4,10_DH2_2,Artefacts,NaN,NaN,DH,NaN,NaN,NaN,DH2,19.98,0.14,60.96,44.95,620.082431,464137.82,22.59,796.21,383.119348,130.52,1.22,31.2375,2.44,5.92,1.26,62.16,0.08,0.93,1.23,14.59,0.76,0.81,0.43,0.735373,4.44,0.27,1.99,0.680000,0.05,0.090000,0.01,0.17,0.07,14.40,0.48,0.80,0.12,0.51,0.09,0.04,0.03,0.01,0.06,0.01,0.02,0.01,0.03,0.00,0.07,0.03,0.69,0.15,0.090000,None,1


In [32]:
my_data_mod = my_data[my_data['Geology'] != 'Artefacts']

In [33]:
my_data_mod_final = pd.concat([my_data_mod, artefact_data], axis = 0).reset_index(drop = True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [34]:
def inlier_label(row):
    if row['Geology'] == 'Bedrock':
        return(row['class'])
    elif row['Geology'] == 'Superficial':
        return(row['class'])
    else:
        return(row['inlierLabel'])
my_data_mod_final['class'] = my_data_mod_final.swifter.apply(inlier_label, axis = 1)

Pandas Apply: 100%|██████████| 1596/1596 [00:00<00:00, 28627.25it/s]


In [35]:
my_data_mod_final.tail()

,Al27,Analysis,As75,B11,Ba137,Band,Be9,Ca42,Cd111,Ce140,Co59,Cr52,Cs133,Cu63,Dy163,Er166,Eu153,Fe56,Formation,Ga69,Gd157,Ge72,Geology,Hf178,Ho165,In115,K39,La139,Li7,Lu175,Mg24,Mn55,Mo95,Nb93,Nd146,Ni60,Nodule,P31,Pb208,Pr141,Province,Rb85,Region,S33,Sc45,Si28,Site,Sm147,Sn118,Sr88,SubSite,Ta181,Tb159,Th232,Ti47,Tm169,U238,V51,Y89,Yb172,Zn68,Zr90,class,inlierLabel
1591,1347.97,44_MDG367_2,0.54,65.51,5.34765,NaN,0.75,246.61,0.42,0.85,0.23,6.96,0.05,0.44,0.13,0.05,0.04,161.72,NaN,0.524997,0.10,1.14,Artefacts,0.010000,0.02,0.01,669.22,0.75,8.37,0.000000,32.90,0.59,0.08,0.31,0.73,2.35,MDG367,144.94,0.34,0.18,NaN,1.64,NaN,773.07,0.61,464405.61,Madawg,0.09,0.37,12.48,NaN,0.03,0.03,0.170000,36.79,0.000000,0.11,1.54,0.50,0.010000,8.11,2.230000,-1,-1.0
1592,1339.24,45_MDG367_3,0.60,55.05,28.69000,NaN,0.16,287.43,0.15,0.65,0.19,3.61,0.04,0.44,0.08,0.04,0.03,116.81,NaN,0.610000,0.14,1.00,Artefacts,0.050000,0.01,0.01,662.40,0.57,10.77,0.010000,32.76,0.71,0.07,0.26,0.58,1.53,MDG367,125.81,0.36,0.14,NaN,1.64,NaN,742.93,0.63,464611.17,Madawg,0.08,0.25,3.93,NaN,0.03,0.02,0.160000,57.23,0.010000,0.12,1.43,0.52,0.030000,8.75,1.370000,1,1.0
1593,512.46,46_MDG370_1,0.61,58.17,1.61000,NaN,0.68,269.73,0.10,0.64,0.17,2.37,0.02,0.54,0.07,0.04,0.05,49.28,NaN,0.320000,0.15,0.80,Artefacts,0.050000,0.03,0.03,307.63,0.78,3.05,0.000000,23.46,0.63,0.10,0.24,0.63,1.27,MDG370,198.53,0.26,0.15,NaN,0.67,NaN,689.45,0.57,465572.56,Madawg,0.16,0.18,2.62,NaN,0.01,0.02,0.120000,82.86,0.010000,0.20,1.85,0.80,0.040000,8.96,0.970000,1,1.0
1594,549.53,47_MDG370_2,0.66,67.16,1.80000,NaN,0.99,266.44,0.17,0.64,0.19,5.54,0.02,0.28,0.42,0.26,0.06,44.38,NaN,0.250000,0.23,0.87,Artefacts,0.043358,0.08,0.02,321.96,0.77,3.06,0.018164,25.17,0.57,0.07,0.11,0.49,0.62,MDG370,168.92,0.22,0.17,NaN,0.61,NaN,808.44,0.59,465382.48,Madawg,0.15,0.20,2.42,NaN,0.02,0.06,0.102174,17.92,0.019198,0.66,0.85,2.56,0.074787,8.45,1.298113,1,1.0
1595,532.25,48_MDG370_3,0.62,62.34,2.21000,NaN,0.99,232.31,0.05,0.58,0.20,5.65,0.02,0.28,0.08,0.08,0.01,46.68,NaN,0.370000,0.06,0.88,Artefacts,0.060000,0.02,0.01,330.35,0.75,3.04,0.010000,25.49,0.62,0.09,0.08,0.61,1.25,MDG370,188.86,0.25,0.16,NaN,0.65,NaN,802.52,0.50,465456.06,Madawg,0.22,0.15,2.57,NaN,0.00,0.02,0.080000,28.52,0.010000,0.24,0.68,0.77,0.040000,7.09,1.380000,1,1.0


In [37]:
#tsne_df_train_labelled = pd.concat([tsne_df_train.reset_index(drop = True), train_data['class'].reset_index(drop = True)], axis = 1)
#tsne_df_bedrock_train_labelled = pd.concat([tsne_df_bedrock_train.reset_index(drop = True), bedrock_data['class'].reset_index(drop = True)], axis = 1)
#tsne_df_superficial_train_labelled = pd.concat([tsne_df_superficial_train, superficial_data['class']], axis = 1)
#tsne_df_test['class'] = 1
tsne_df_everything_labelled = pd.concat([tsne_df_everything.reset_index(drop = True), my_data_mod_final['class'].reset_index(drop = True)], axis = 1)

In [38]:
PC_df_train_labelled = pd.concat([PC_df_train.reset_index(drop = True), train_data['class'].reset_index(drop = True)], axis = 1)
PC_df_superficial_train_labelled = pd.concat([PC_df_superficial_train.reset_index(drop = True), superficial_data['class']], axis = 1)
PC_df_bedrock_train_labelled = pd.concat([PC_df_bedrock_train.reset_index(drop = True), bedrock_data['class'].reset_index(drop = True)], axis = 1)
PC_df_everything_labelled = pd.concat([PC_df_everything.reset_index(drop = True), my_data_mod_final['class'].reset_index(drop = True)], axis = 1)

In [43]:
tsne_df_everything_labelled.tail()

,tsne1,tsne2,tsne3,class
1591,8.222127,30.328259,-1.500060,-1
1592,19.752899,35.494946,-4.201612,1
1593,-10.546824,12.391637,13.616352,1
1594,-19.075029,-25.947941,19.238798,1
1595,-11.559745,-16.622896,-10.937040,1


In [40]:
my_data['class'].unique()

array(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG',
       'WB_BX', 'PF', 'BM', 'WH', 'SQ', 'BP', 'WN', 'BH', 'PH', 'LB',
       'AB', 'LV', 'SV_SE', 'BA', 'WA', 'MM', 'Artefact'], dtype=object)

In [44]:
#tsne_df_train_labelled.to_csv('tsne_both.csv')
#tsne_df_bedrock_train_labelled.to_csv('tsne_bedrock.csv')
#tsne_df_superficial_train_labelled.to_csv('tsne_superficial.csv')
#tsne_df_test.to_csv('tsne_artefacts.csv')
#tsne_df_everything_labelled.to_csv('tsne_samples_and_artefacts.csv')

In [42]:
#PC_df_train_labelled.to_csv('pca_both.csv')
#PC_df_superficial_train_labelled.to_csv('pca_superficial.csv')
#PC_df_bedrock_train_labelled.to_csv('pca_bedrock.csv')
#PC_df_everything_labelled.to_csv('pca_samples_and_artefacts.csv')